# Experiment README

## Table of Contents

* [Overview of Experiment Architecture](#Overview-of-Experiment-Architecture)
* [Experiment Workflow](#Experiment-Workflow)
    * [Modifying State Variables](#Modifying-State-Variables)
    * [Modifying System Parameters](#Modifying-System-Parameters)
    * [Executing Experiments](#Executing-Experiments)
    * [Post-processing and Analysing Results](#Post-processing-and-Analysing-Results)
    * [Visualizing Results](#Visualizing-Results)
* [Creating New, Customized Experiment Notebooks](#Creating-New,-Customized-Experiment-Notebooks)
    * Step 1: Select an experiment template
    * Step 2: Create a new notebook
    * Step 3: Customize the experiment
    * Step 4: Execute the experiment
* [Advanced Experiment-configuration & Simulation Techniques](#Advanced-Experiment-configuration-&-Simulation-Techniques)
    * [Setting Simulation Timesteps and Unit of Time `dt`](#Setting-Simulation-Timesteps-and-Unit-of-Time-dt)
    * [Changing the Ethereum Network Upgrade Stage](#Changing-the-Ethereum-Network-Upgrade-Stage)
    * [Performing Large-scale Experiments](#Performing-Large-scale-Experiments)

# Overview of Experiment Architecture

The experiment architecture is composed of the following four elements – the **model**, **default experiment**, **experiment templates**, and **experiment notebooks**:

1. The **model** is initialized with a default Initial State and set of System Parameters defined in the `model` module.
2. The **default experiment** – in the `experiments.default_experiment` module – is an experiment composed of a single simulation that uses the default cadCAD **model** Initial State and System Parameters. Additional default simulation execution settings such as the number of timesteps and runs are also set in the **default experiment**.
3. The **experiment templates** – in the `experiments.templates` module – contain pre-configured analyses based on the **default experiment**. Examples include... To be created!
4. The **experiment notebooks** perform various scenario analyses by importing existing **experiment templates**, optionally modifying the Initial State and System Parameters within the notebook, and then executing them.

# Experiment Workflow

If you just want to run (execute) existing experiment notebooks, simply open the respective notebook and execute all cells.

Depending on the chosen template and planned analysis, the required imports might differ slightly from the below standard dependencies:

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook-configuration methods, such as loading IPython modules
import setup

# External dependencies
import copy
import logging
import pandas as pd
import plotly.express as px
from pprint import pprint
import importlib as imp

# Project dependencies
import model.constants as constants
from experiments.run import run
from experiments.utils import display_code
import experiments.visualizations as visualizations

In [ ]:
#import dask.dataframe as dd
import pandas as pd

In [ ]:
#%time
#df_dd = dd.read_parquet('../../data/mock_logreturns.prq')
#df_pd = pd.read_parquet('../../data/mock_logreturns.prq')
#df_pd = pd.read_csv('../../data/mock_logreturns.csv')

We can then import the default experiment, and create a copy of the simulation object – we create a new copy for each analysis we'd like to perform:

In [ ]:
import experiments.default_experiment as default_experiment
simulation_analysis_1 = copy.deepcopy(default_experiment.experiment.simulations[0])

We can use the `display_code` method to see the configuration of the default experiment before making changes:

In [ ]:
#display_code(default_experiment)  # In this example equivalent to display_code(simulation_analysis_1.)

Alternatively to modifying the default experiment in a notebook as shown in the next section, we can also load predefined experiment templates: 

In [ ]:
#import experiments.templates.monte_carlo_analysis as monte_carlo_analysis
#simulation_analysis_2 = copy.deepcopy(monte_carlo_analysis.experiment.simulations[0])
#display_code(monte_carlo_analysis)

## Modifying State Variables

To view what the Initial State (radCAD model-configuration setting `initial_state`) of the State Variables are, and to what value they have been set, we can inspect the dictionary as follows:

In [ ]:
pprint(simulation_analysis_1.model.initial_state)

In [ ]:
simulation_analysis_1.model.state

To modify the value of **State Variables** for a specific analysis, you need to select the relevant simulation and update the chosen model Initial State. For example, updating the `floating_supply` Initial State to `100e6` CELO and `123e5` cUSD.

In [ ]:
simulation_analysis_1.model.initial_state.update({
    'reserve_balance': {
        'celo': 120.0e6,
        'cusd': 0.0},
})

In [ ]:
pprint(simulation_analysis_1.model.initial_state)

## Modifying System Parameters

To view what the System Parameters (radCAD model configuration setting `params`) are, and to what value they have been set, we can inspect the dictionary as follows:

In [ ]:
pprint(simulation_analysis_1.model.params)

To modify the value of **System Parameters** for a specific analysis, you need to select the relevant simulation, and update the chosen model System Parameter (which is a list of values). For example, updating the `reserve_fraction` System Parameter to a sweep of two values, `0.001` and `0.01`:

In [ ]:
simulation_analysis_1.model.params.update({
    "reserve_fraction": [0.001, 0.01],
})

## Executing Experiments

We can now execute our custom analysis and retrieve the post-processed Pandas DataFrame using the `run(...)` method:

In [ ]:
df, exceptions = run(simulation_analysis_1)
#%lprun -T lprof0 -f mp.p_market_price run(simulation_analysis_1)

## Post-processing and Analysing Results

We can see that we had no exceptions for the single simulation we executed:

In [ ]:
df[df['subset']==0].head()

In [ ]:
df[df['subset']==0].tail()

In [ ]:
exceptions[0]['exception'] == None

We can simply display the Pandas DataFrame to inspect the results. This DataFrame already has some default post-processing applied (see [experiments/post_processing.py](../post_processing.py)). For example, parameters that change in the parameter grid (if there are any) are attached as columns to the end of the dataframe.

In [ ]:
# Show which reserve_fraction values were used in the grid
df.groupby('subset')['reserve_fraction'].unique()

We can also use Pandas for numerical analyses:

In [ ]:
# Get the maximum mento_rate for each subset: in this example each reserve_fraction value used in the grid.
df.groupby('subset')['oracle_rate'].max()

## Visualizing Results

Once we have the results post-processed and in a Pandas DataFrame, we can use Plotly for plotting our results (here two subsets because of the `reserve_fraction` parameter sweep introduced above):

In [ ]:
# Plot the mento_rate for each subset (each parameter grid combination) directly from the df (not preferred)
visualizations.plot_oracle_rate(df)

In [ ]:
df[(df['subset']==0) & (df['run']==1)].head(20)

In [ ]:
# Or use a respective visualizations predefined in the visualizations module (preferred)
visualizations.plot_celo_market_price(df)

In [ ]:
visualizations.plot_reserve_balance(df)

In [ ]:
visualizations.plot_cusd_market_price(df)